In [ ]:
import os
import numpy as np
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation, advanced_activations, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from keras import regularizers, optimizers
from keras.optimizers import SGD
from keras.models import Model
#import GlobalAvgPool2D
from sklearn.preprocessing import LabelEncoder
from skimage import io, transform
from sklearn.metrics import accuracy_score
from scipy import misc
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Input,Concatenate
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import random
#layer1 32*32*3
def vgg16(num_classes):
    model = Sequential()
    #第一个 卷积层 的卷积核的数目是32 ，卷积核的大小是3*3，stride没写，默认应该是1*1
    #对于stride=1*1,并且padding ='same',这种情况卷积后的图像shape与卷积前相同，本层后shape还是32*32
    weight_decay=0.0005
    model.add(Conv2D(64, (3, 3), padding='same',input_shape=(IMAGE_SIZE,IMAGE_SIZE,3),kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    #进行一次归一化
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    #layer2 32*32*64
    model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    #下面两行代码是等价的，#keras Pool层有个奇怪的地方，stride,默认是(2*2),
    #padding默认是valid，在写代码是这些参数还是最好都加上,这一步之后,输出的shape是16*16*64
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding='same')  )
    #layer3 16*16*64
    model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    #layer4 16*16*128
    model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #layer5 8*8*128
    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    #layer6 8*8*256
    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    #layer7 8*8*256
    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #layer8 4*4*256
    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    #layer9 4*4*512
    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    #layer10 4*4*512
    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #layer11 2*2*512
    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    #layer12 2*2*512
    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    #layer13 2*2*512
    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    #layer14 1*1*512
    model.add(Flatten())
    model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    #layer15 512
    model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    #layer16 512
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    # 10
    # model.summary()
    return model

#model.save('my_model_bp.h5')
def loadData(IMAGE_SIZE):
    images = os.listdir(DATA_DIR)
    d = {}
    train_data = []
    train_pics=[]
    test_pics=[]
    test_labels=[]
    train_labels = []
    train_data_all=[]
    train_pics_all=[]
    train_labels_all = []
    for idx, dir in enumerate(os.listdir(DATA_DIR)):
            d[idx] = dir
            label = idx
            subdir = os.path.join(DATA_DIR, dir)
            for i, file in enumerate(os.listdir(subdir)):
                pic = misc.imread(os.path.join(subdir, file))
                pic = misc.imresize(pic, (IMAGE_SIZE,IMAGE_SIZE, 3))
                train_data_all.append((pic, label))
    np.random.shuffle(train_data_all)
    validation_size = int(len(train_data_all) * 0.20)
    test_data = train_data_all[:validation_size].copy()
    np.random.shuffle(test_data)
    train_data = train_data_all[validation_size:]
    for i in range(len(train_data)):
        train_pics.append(train_data[i][0])
        train_labels.append(train_data[i][1])
    for i in range(len(test_data)):
        test_pics.append(test_data[i][0])
        test_labels.append(test_data[i][1])
    for i in range(len(train_data_all)):
        train_pics_all.append(train_data_all[i][0])
        train_labels_all.append(train_data_all[i][1])
    return np.array(train_pics), np.array(train_labels),np.array(test_pics), np.array(test_labels),np.array(train_pics_all), np.array(train_labels_all)

np.random.seed(25)
#加载数据
IMAGE_SIZE = 227
DATA_DIR = r"G:\科研\数据集\opendata_VRID\model"
train_data, train_labels, test_data, test_labels,train_data_all,train_labels_all = loadData(IMAGE_SIZE)
#输出训练和测试数据量
# print ("Train data size: ", len(train_data))
# print ("Test data size: ", len(test_data))
#将标签量进行转化
train_labels=np_utils.to_categorical(train_labels)
test_labels=np_utils.to_categorical(test_labels)
train_labels_all=np_utils.to_categorical(train_labels_all)
#设置检查点
# filepath=r"G:\科研\数据集\opendata_VRID\model\weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]
#设置最佳检查点
filepath=r"G:\科研\数据集\opendata_VRID\weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
#设置模型
num_classes=10
model=vgg16(num_classes)
#model=squeezenet(227,227,3,num_classes)
#编译模型
epochs = 10
learning_rate = 0.01
#decay_rate = learning_rate / epochs
momentum = 0.9
sgd = SGD(lr=learning_rate, momentum=momentum,  decay=1e-6, nesterov=False)
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
#sgd = SGD(lr=0.0, momentum=0.9, decay=0.0, nesterov=False)
#model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
#匹配模型
#lrate = LearningRateScheduler(step_decay)
#callbacks_list = [lrate]
history=model.fit(train_data_all, train_labels_all,validation_split=0.2, nb_epoch=epochs,batch_size=64,callbacks=callbacks_list)
#测试模型
preds = np.argmax(model.predict(test_data), axis=1)
test_labels = np.argmax(test_labels, axis=1)
print (accuracy_score(test_labels, preds))
#保存模型
model.save('Vgg16net-model(10,64).model')
#记录模型日志
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
#画模型准确率曲线和损失率曲线
plt.plot(epochs, acc, 'ro:', label='Training acc')
plt.plot(epochs, val_acc, 'bo:', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'ro:', label='Training loss')
plt.plot(epochs, val_loss, 'bo:', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()
plt.savefig('graph.png')
